In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/food-recommendation-system/1662574418893344.csv
/kaggle/input/food-recommendation-system/ratings.csv


This will be a very simple model. I will take the ratings, construct a cosine similarity matrix and output predictions. 

In [2]:
ratings = pd.read_csv('/kaggle/input/food-recommendation-system/ratings.csv')

We can map the food ids later. 

In [3]:
ratings = ratings.sample(frac=1, random_state=42).reset_index(drop=True)

In [4]:
ratings.columns

Index(['User_ID', 'Food_ID', 'Rating'], dtype='object')

In [5]:
user_item_matrix = ratings.pivot_table(
    index='User_ID',
    columns='Food_ID',
    values='Rating',
    fill_value=0
)

This constructs a matrix where the rows are the user values, the columns are the food items and the values in the matrix are the corresponding ratings at that location. 

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

item_vectors = user_item_matrix.T

item_similarity_matrix = cosine_similarity(item_vectors)

In [7]:
def get_similar_items(food_id, top_n=5):
    item_index = item_vectors.index.get_loc(food_id)
    similarity_scores = item_similarity_matrix[item_index]
    similar_indices = np.argsort(similarity_scores)[::-1][1:top_n+1]
    similar_food_ids = item_vectors.index[similar_indices].tolist()
    return similar_food_ids

And that is it. What cosine similarity does is it takes the columns in the matrix (which represent food items) as vectors. Then, it calculates the angle between this vector and other vectors (food items) and returns the column with the lowest angle, which suggests that column had the most similar ratings from users. 

In [10]:
real = pd.read_csv('/kaggle/input/food-recommendation-system/1662574418893344.csv')

In [11]:
real.columns

Index(['Food_ID', 'Name', 'C_Type', 'Veg_Non', 'Describe'], dtype='object')

In [14]:
from sklearn.preprocessing import LabelEncoder

food_labels = real[['Food_ID', 'Name']].drop_duplicates()


name_to_id = dict(zip(food_labels['Name'].str.lower(), food_labels['Food_ID']))

id_to_name = dict(zip(food_labels['Food_ID'], food_labels['Name']))

le = LabelEncoder()
le.fit(food_labels['Name'].str.lower())

print(name_to_id)

{'summer squash salad': 1, 'chicken minced salad': 2, 'sweet chilli almonds': 3, 'tricolour salad': 4, 'christmas cake': 5, 'japanese curry arancini with barley salsa': 6, 'chocolate nero cookies': 7, 'lamb and chargrilled bell pepper soup': 8, 'cream of almond soup': 9, 'broccoli and almond soup': 10, 'coconut lime quinoa salad': 11, 'lemon honey glazed sous vide corn on the cob': 12, 'watermelon and strawberry smoothie': 13, 'peach, raspberry and nuts smoothie': 14, 'almond and cranberry poha': 15, 'almond and raw banana galawat': 16, 'baked namakpara with roasted almond dip': 17, 'grilled almond barfi': 18, 'baked shankarpali ': 19, 'baked multigrain murukku': 20, 'apple rabdi': 21, 'baked namak para': 22, 'dates and nuts ladoo': 23, 'green lentil dessert fudge': 24, 'cashew nut cookies': 25, 'almond pearls': 26, 'hawaiin papaya salad': 27, 'vegetable som tam salad': 28, 'spinach and feta crepes': 29, 'couscous with ratatouille - tangy tomato sauce': 30, 'baked almond kofta': 31, 'a

In [15]:
image_to_id = {
    'apple_pie': 225,
    'baby_back_ribs': 328,
    'baklava': None,
    'beef_carpaccio': None,
    'beef_tartare': None,
    'beet_salad': 38,
    'beignets': None,
    'bibimbap': 368,
    'bread_pudding': 266,
    'breakfast_burrito': None,
    'bruschetta': 184,
    'caesar_salad': 1,
    'cannoli': None,
    'caprese_salad': 4,
    'carrot_cake': 207,
    'ceviche': 143,
    'cheesecake': 228,
    'cheese_plate': 248,
    'chicken_curry': 100,
    'chicken_quesadilla': 44,
    'chicken_wings': 113,
    'chocolate_cake': 276,
    'chocolate_mousse': 215,
    'churros': None,
    'clam_chowder': None,
    'club_sandwich': 193,
    'crab_cakes': None,
    'creme_brulee': None,
    'cupcakes': 230,
    'deviled_eggs': 288,
    'donuts': 206,
    'dumplings': 377,
    'edamame': None,
    'eggs_benedict': 271,
    'escargots': None,
    'falafel': None,
    'filet_mignon': None,
    'fish_and_chips': 159,
    'foie_gras': None,
    'french_fries': None,
    'french_onion_soup': 8,
    'french_toast': None,
    'fried_calamari': None,
    'fried_rice': 166,
    'garlic_bread': 314,
    'gnocchi': 318,
    'greek_salad': 70,
    'grilled_cheese_sandwich': 286,
    'guacamole': None,
    'gyoza': None,
    'hamburger': 356,
    'hot_and_sour_soup': None,
    'hot_dog': None,
    'huevos_rancheros': None,
    'hummus': None,
    'ice_cream': 68,
    'lasagna': 109,
    'lobster_bisque': None,
    'lobster_roll_sandwich': None,
    'macarons': 220,
    'miso_soup': 312,
    'mussels': None,
    'nachos': None,
    'omelette': None,
    'onion_rings': None,
    'oysters': 129,
    'pad_thai': 372,
    'paella': 175,
    'pancakes': 227,
    'panna_cotta': None,
    'peking_duck': None,
    'pho': 322,
    'pizza': 46,
    'pork_chop': 50,
    'prime_rib': None,
    'pulled_pork_sandwich': None,
    'ramen': 338,
    'ravioli': None,
    'red_velvet_cake': 266,
    'risotto': 165,
    'samosa': 279,
    'sashimi': None,
    'scallops': None,
    'seaweed_salad': None,
    'shrimp_and_grits': 145,
    'spaghetti_bolognese': 190,
    'spaghetti_carbonara': 287,
    'spring_rolls': 223,
    'steak': 313,
    'strawberry_shortcake': 185,
    'sushi': 205,
    'tacos': 397,
    'takoyaki': None,
    'tiramisu': None,
    'tuna_tartare': None,
    'waffles': None
}

In [16]:
def namerecommendation(foodname, top_n=5):
    identity = name_to_id[foodname]
    ans = id_to_name[get_similar_items(identity, top_n)]
    return ans

In [17]:
def imgtorec(imgname, top_n):
    idenitity = image_to_id[imgname]
    ans = ans = id_to_name[get_similar_items(identity, top_n)]
    return ans